# Creating a custom chatbot & feeding context using OpenAI API - includes image generation

In [13]:
from openai import OpenAI
import panel as pn
import base64
import io
from PIL import Image

client = OpenAI(api_key='API K')

# defining functions that we'll use to call OpenAI API
def get_response_from_messages(messages, model="gpt-4o", temperature = 0.2):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature 
    )
    return response.choices[0].message.content

In [14]:
def generate_image(prompt):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url
    return image_url

In [17]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    
    #Always add the User's message to the panels first
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, styles={'background-color': '#25E9E3'})))
    
    context.append({'role':'user','content':f"{prompt}"})
    
   # Check if the user is asking for an image
    if any(phrase in prompt.lower() for phrase in ['show image', 'show me an image', 'create image', 'generate image', 'show me']):
        # Check if the previous user message was also asking for an image
        if len(context) > 2 and any(phrase in context[-3]['content'].lower() for phrase in ['show image', 'show me an image', 'create image', 'generate image', 'show me']):
            response = "I've already shown you an image for the last explanation. Could you ask a new question so I can provide a fresh explanation?"
        elif len(context) > 2:
            previous_explanation = context[-3]['content']
            image_prompt = f"Create a simple, child-friendly illustration based on this: {previous_explanation}"
            image_url = generate_image(image_prompt)
            
            response = "Here's an image. Is there anything else you'd like to know?"
            
            # Add the Assistant's response to the panels
            panels.append(
                pn.Row('Assistant:', pn.pane.Markdown(response, styles={'background-color': '#D7E925'})))
            
            # Then add the image
            panels.append(pn.Row('Image:', pn.pane.Image(image_url, width=400, height=400)))
            
            # Early return to avoid adding the response twice
            context.append({'role':'assistant','content':f"{response}"})
            return pn.Column(*panels)
        else:
            response = "I'm sorry, but I don't have any previous explanation to base an image on. Could you ask a question first?"
    else:
        response = get_response_from_messages(context)
    
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, styles={'background-color': '#D7E925'})))
    
    context.append({'role':'assistant','content':f"{response}"})
    
    return pn.Column(*panels)

In [18]:
pn.extension()

panels = [] #collecting display

context = [ {'role':'system', 'content': """ You are a teaching assistant exclusively meant to help children understand complex topics in a simple manner. \
             All your answers will be completely free of harmful and hateful content. \
             You will first greet the child cheerfully, then ask for what they want to learn. \
             Use different child-friendly emojis based on the context. \
             You will then answer their question in the simplest manner, in the format of Explain Like I'm 5 (ELI5). \ 
             Limit your answer to less than 200 words. \
             Once you finish answering, make sure to ask if there are any follow-up questions. \ """} ] 
#feeding context to the System (chatbot), 
#which is kinda like the brain of the custom chatbot we are building

inp = pn.widgets.TextInput(value="Hi", placeholder = 'Enter text here...')
button = pn.widgets.Button(name= "Chat!")
interactive_conversation = pn.bind(collect_messages, button)

dashboard = pn.Column(
    inp,
    pn.Row(button),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


Column
    [0] TextInput(placeholder='Enter text here...')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)